In [1]:
text = open('tinyshakespeare.txt').read()
len(text)

1115394

In [2]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [3]:
# tokenize
stoi = {ch: i for i,ch in enumerate(chars)}
itos = {i: ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode('hi there'))
print(decode(encode('hi there')))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [4]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [5]:
# Let's split up the train and validation dataset
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

len(train_data), len(val_data)

(1003854, 111540)

In [6]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [7]:
torch.manual_seed(1337)
batch_size = 4           # how many individual sequences to process in parallel
block_size = 8           # how many maximum individual characters in the example, max context length

def get_batch(split, batch_size=batch_size):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) 
    return x, y

In [8]:
xb, yb = get_batch('train')

In [9]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'{context=}, {target=}')

context=tensor([24]), target=tensor(43)
context=tensor([24, 43]), target=tensor(58)
context=tensor([24, 43, 58]), target=tensor(5)
context=tensor([24, 43, 58,  5]), target=tensor(57)
context=tensor([24, 43, 58,  5, 57]), target=tensor(1)
context=tensor([24, 43, 58,  5, 57,  1]), target=tensor(46)
context=tensor([24, 43, 58,  5, 57,  1, 46]), target=tensor(43)
context=tensor([24, 43, 58,  5, 57,  1, 46, 43]), target=tensor(39)
context=tensor([44]), target=tensor(53)
context=tensor([44, 53]), target=tensor(56)
context=tensor([44, 53, 56]), target=tensor(1)
context=tensor([44, 53, 56,  1]), target=tensor(58)
context=tensor([44, 53, 56,  1, 58]), target=tensor(46)
context=tensor([44, 53, 56,  1, 58, 46]), target=tensor(39)
context=tensor([44, 53, 56,  1, 58, 46, 39]), target=tensor(58)
context=tensor([44, 53, 56,  1, 58, 46, 39, 58]), target=tensor(1)
context=tensor([52]), target=tensor(58)
context=tensor([52, 58]), target=tensor(1)
context=tensor([52, 58,  1]), target=tensor(58)
context=t

In [10]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [11]:
# lets start with a bigram language model

In [12]:
import torch.nn as nn
from torch.nn import functional as F

In [13]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from the embedding table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # bigram table
        
    def forward(self, idx, targets=None):
        #idx and targets are both (B,T) tensors of integers
        logits = self.token_embedding_table(idx) # (B, T, C) batch, time, channels
        
        if targets is None:
            loss = None
        else:        
            B, T, C = logits.shape
            logits = logits.view(B*T, C) 
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # id is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # get probs
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequenc
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [14]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(loss)

tensor(4.8786, grad_fn=<NllLossBackward0>)


In [15]:
# since 65 is the vocab we can guess what the loss should be
# probs = 1/65, -ln(probs) = 
# -logn(probs)

In [16]:
-torch.log(torch.tensor(1/65))

tensor(4.1744)

In [17]:
# the initial predictions are not well diffused, equally likely to begin with

In [18]:
torch.zeros((1, 1), dtype=torch.long)

tensor([[0]])

In [19]:
# 0 is going to be how we kick off the generation

In [20]:
print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [21]:
m = BigramLanguageModel(vocab_size)

In [22]:
# pytorch optimizer
# optimizer : take the gradients and updates the parameters
# quite good optimizer, typical good settings is 3e-4
# but for small networks like this here
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [23]:
batch_size = 32
steps = 500000
for step in range(steps):
    xb, yb = get_batch('train')
    
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    if (step+1)%(steps/10) == 0:
        print(f'Step: {step} | Loss: {loss.item()}')

print(f'Final loss : {loss.item()}')

Step: 49999 | Loss: 2.216499090194702
Step: 99999 | Loss: 2.4571011066436768
Step: 149999 | Loss: 2.4942376613616943
Step: 199999 | Loss: 2.4318201541900635
Step: 249999 | Loss: 2.4122331142425537
Step: 299999 | Loss: 2.4120142459869385
Step: 349999 | Loss: 2.6919662952423096
Step: 399999 | Loss: 2.2373297214508057
Step: 449999 | Loss: 2.2734100818634033
Step: 499999 | Loss: 2.567037582397461
Final loss : 2.567037582397461


In [24]:
print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=200)[0].tolist()))


PULOugishamy cre hitayoromyshim.
ILYot nd I m fu mes fayof tike,-hid tllarpon h, t I oullmoplofoveing thouse wr's o mobe
Whoour, do, t beous ore:
BUKI y dyoll'dvan otou s g ouk mus. oharar;



D an! h


In [25]:
# Dramatic improvement already
# Certainly not shakespeare but the model's making progress

In [26]:
# Tokens are not talking to each other yet, this is the most basic language model and it's not too bad
# regardless of the context passed in, it only looks at the last character in the context
# The current model doesn't really care about it, in the next nb, we'll make use of context